In [ ]:
from azure.identity import AzureCliCredential
from azure.core.credentials import TokenCredential, AccessToken
from requests import get
import time
import logging
logger = logging.getLogger(__name__)

def calc_time(string):
    global start_time

    if string == "start":
        start_time = time.time()
    else:
        end_time = time.time()
        return  end_time - start_time


class CachedCredential(TokenCredential):
  def __init__(self, delegate: TokenCredential, logger) -> None:
    self.delegate = delegate
    self.logger = logger
    self._token : dict[str, AccessToken] = {}

  def get_token(self, scope: str, **kwargs) -> AccessToken:
    token = self._token.get(scope)
    if not token or token.expiry < time.time():
      calc_time("start")
      self._token[scope] = token = self.delegate.get_token(scope, **kwargs)
      elapsed_time = calc_time("end")
      self.logger.info(
            f"Time taken to generate token(CachedCredential) is {elapsed_time:.2f} seconds."
        )
    else:
        self.logger.info(
            f"Valid token exists"
        )
    return token


cachedCredential = CachedCredential(AzureCliCredential(), logger)

def request_fmc_token(organization_name, stage='prod'):
    """
    Get fmc token via AzureCliCredential. (Requires az login beforehand).

    organization_name e.g. nrcs-2-pf, ford-dat-3, uss-gen-6-pf
    """
    return cachedCredential.get_token(f'api://api-data-loop-platform-{organization_name}-{stage}/.default').token


def get_sequences(fmc_query, organization_name, fmc_token):
    """
    Does get sequences Rest call for fmc query. Returns list of sequences.
    """
    fmc_headers = {
        'Cache-Control': 'no-cache',
        'Authorization': f'Bearer {fmc_token}',
        'Origin': 'https://developer.bosch-data-loop.com'
    }

    sequences = []

    items_per_page = 1000

    is_there_more_sequences = True
    page_index = 0
    while is_there_more_sequences:
        url = f'https://api.azr.bosch-data-loop.com/measurement-data-processing/v3/organizations/{organization_name}/sequence?itemsPerPage={items_per_page}&pageIndex={page_index}&filterQuery={fmc_query}'  # noqa: E501
        response = get(url, headers=fmc_headers)
        if response.status_code == 200:
            response_sequences = response.json()
            sequences.extend(response_sequences)
            if len(response_sequences) < items_per_page:
                is_there_more_sequences = False
        else:
            logger.error(f'Get sequences call to FMC failed. status_code: {response.status_code}, reason: {response.reason}, url: {url}')
            is_there_more_sequences = False
        page_index += 1
        print(f'FMC query at {page_index=}')

    return sequences


organization_name = 'nrcs-2-pf'
fmc_token = request_fmc_token(organization_name)
fmc_query = 'SequenceCollection.id = "811"'
fmc_sequences = get_sequences(fmc_query, organization_name, fmc_token)

print(f'Found {len(fmc_sequences)} sequences.')

def get_fmc_bolf_path(sequence) -> str | None:
    """Extract the BOLF path from a sequence."""
    for label_file in sequence['labelFiles']:
        if label_file['domain'] == 'KPI_GENERATED_BOLF':
            return label_file['path']
    print(f'Warn: {sequence["id"]} has no bolf linked')
    return None

def bolf_url_to_path(url: str):
    qua_url = 'https://dypersiaqua.blob.core.windows.net/'
    dev_url = 'https://dypersiadev.blob.core.windows.net/'

    # TODO: update to use WriteOnly
    qua_path = '/home/jovyan/data/ReadWrite/dypersiaqua/'
    dev_path = '/home/jovyan/data/ReadWrite/dypersiadev/'

    for blob_url, blob_path in [(qua_url, qua_path), (dev_url, dev_path)]:
        if url.startswith(blob_url):
            return blob_path + url[len(blob_url):]
    
    raise Exception('Unknown url path', url)

bolf_links = [get_fmc_bolf_path(s) for s in fmc_sequences]
bolf_links = [i for i in bolf_links if i]

bolf_paths = [bolf_url_to_path(s) for s in bolf_links]


import json
import os
import shutil
from shapely import Polygon

BUFFER_M = 0.15
DRY_RUN = True

bolf_backup_dir = 'bolf_backup'
os.makedirs(bolf_backup_dir, exist_ok=True)

new_bolfs_dir = 'new_bolfs'
os.makedirs(new_bolfs_dir, exist_ok=True)

def update_bolf(bolf, bolf_filename):
    for frame in bolf['openlabel']['frames'].values():
        for object in frame['objects'].values():
            original_position = object['object_data']['poly3d'][0]['val']
            original_xy = [(original_position[i], original_position[i+1]) for i in range(0, len(original_position), 3)]

            if len(original_xy) <= 2:
                print(f'Warn: In {bolf_filename} found label with 1 point. Skipping - but should be fixed by fix script')
                continue

            obstacle_poly = Polygon(original_xy)
            obstacle_poly = obstacle_poly.buffer(BUFFER_M, cap_style='round', join_style='mitre', mitre_limit=2.0, quad_segs=3)
            buffered_position = [coord for x, y in obstacle_poly.exterior.coords for coord in (x, y, None)][:-3] 
            object['object_data']['poly3d'][0]['val'] = buffered_position
    
    return bolf


for bolf_path in bolf_paths:
    bolf_filename = os.path.basename(bolf_path)
    shutil.copy(bolf_path, bolf_backup_dir)
    with open('bolf_backup.txt', 'a+') as f:
        f.write(f'{bolf_path} {bolf_backup_dir}/{bolf_filename}\n')
    
    with open(bolf_path) as f:
        bolf = json.loads(f.read())
    
    bolf = update_bolf(bolf, bolf_filename)
    
    with open(os.path.join(new_bolfs_dir, bolf_filename), 'w') as f:
        f.write(json.dumps(bolf))

    if not DRY_RUN:
        with open(bolf_path, 'w') as f:
            f.write(json.dumps(bolf))
